# Southeast Asian Android Phone Marketplace Analysis
## AI-Powered Analytics Dashboard

In [ ]:
# Install required packages
!pip install openai
!pip install git+https://github.com/openai/swarm.git

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from openai import OpenAI
from swarm import Swarm, Agent
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# Load and preprocess data
df = pd.read_csv('https://raw.githubusercontent.com/strength-programmer/DAY_6/refs/heads/main/Project/ai%20first%20sales%20data%20-%20sales.csv')

# Data preprocessing
def preprocess_data(df):
    df_clean = df.copy()
    numeric_columns = ['revenue', 'transactions', 'pageviews', 'visits', 
                      'productClick', 'addToCart', 'checkout', 'ad_spend']
    
    for col in numeric_columns:
        if col in df_clean.columns:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    
    if 'date' in df_clean.columns:
        df_clean['date'] = pd.to_datetime(df_clean['date'])
    
    return df_clean

df_clean = preprocess_data(df)
print(f"Dataset loaded with {len(df_clean)} rows")

In [ ]:
# Initialize OpenAI client
api = OpenAI(api_key="your-api-key-here")
client = Swarm(api)

# Initialize agents
data_quality_agent = Agent(
    name="Data Quality Agent",
    instructions="""Analyze data quality focusing on:
    1. Missing values and data completeness
    2. Anomalies in metrics
    3. Data type validation
    4. Suspicious patterns"""
)

marketing_agent = Agent(
    name="Marketing Agent",
    instructions="""Analyze marketing performance focusing on:
    1. Channel performance
    2. ROI analysis
    3. Campaign effectiveness"""
)

customer_journey_agent = Agent(
    name="Customer Journey Agent",
    instructions="""Analyze customer journey focusing on:
    1. Conversion funnel
    2. Drop-off points
    3. User behavior patterns"""
)

revenue_agent = Agent(
    name="Revenue Agent",
    instructions="""Analyze revenue patterns focusing on:
    1. Revenue trends
    2. Growth analysis
    3. Performance metrics"""
)

### 1. Data Quality Analysis

In [ ]:
quality_response = client.run(
    agent=data_quality_agent,
    messages=[{
        "role": "user",
        "content": "Analyze the data quality and provide insights about missing values, anomalies, and data types."
    }],
    context_variables={"dataframe": df_clean}
)

print("\nData Quality Analysis:")
print(quality_response.messages[-1].content)

### 2. Marketing Analysis

In [ ]:
# Marketing Analysis
marketing_response = client.run(
    agent=marketing_agent,
    messages=[{"role": "user", "content": "Analyze marketing performance and ROI across channels"}],
    context_variables={"dataframe": df_clean}
)

print("\nMarketing Analysis:")
print(marketing_response.messages[-1].content)

# Marketing visualizations
source_metrics = df_clean.groupby('source').agg({
    'revenue': 'sum',
    'ad_spend': 'sum',
    'transactions': 'sum'
}).reset_index()

# Revenue by Channel
fig1 = px.pie(source_metrics, values='revenue', names='source',
              title='Revenue by Channel')
fig1.show()

# Revenue vs Ad Spend
fig2 = px.bar(source_metrics, x='source', y=['revenue', 'ad_spend'],
              title='Revenue vs Ad Spend by Channel')
fig2.show()

### 3. Customer Journey Analysis

In [ ]:
journey_response = client.run(
    agent=customer_journey_agent,
    messages=[{"role": "user", "content": "Analyze the customer journey and conversion funnel"}],
    context_variables={"dataframe": df_clean}
)

print("\nCustomer Journey Analysis:")
print(journey_response.messages[-1].content)

# Funnel visualization
funnel_data = {
    'Stage': ['Pageviews', 'Product Clicks', 'Add to Cart', 'Checkout', 'Transactions'],
    'Count': [
        df_clean['pageviews'].sum(),
        df_clean['productClick'].sum(),
        df_clean['addToCart'].sum(),
        df_clean['checkout'].sum(),
        df_clean['transactions'].sum()
    ]
}
fig = go.Figure(go.Funnel(y=funnel_data['Stage'], x=funnel_data['Count']))
fig.update_layout(title='Customer Journey Funnel')
fig.show()

### 4. Revenue Analysis

In [ ]:
revenue_response = client.run(
    agent=revenue_agent,
    messages=[{"role": "user", "content": "Analyze revenue trends and patterns"}],
    context_variables={"dataframe": df_clean}
)

print("\nRevenue Analysis:")
print(revenue_response.messages[-1].content)

# Revenue trend visualization
daily_revenue = df_clean.groupby('date')['revenue'].sum().reset_index()
fig = px.line(daily_revenue, x='date', y='revenue',
              title='Daily Revenue Trend')
fig.update_xaxes(type='date')
fig.show()